# Model Training Script
This notebook is a transformation of the provided Python script into a Jupyter notebook. The script is designed for training a VisionEncoderDecoderModel using a custom dataset.

In [1]:
!pip install transformers datasets torch torchvision streamlit fastapi uvicorn accelerate sentencepiece
!pip install tensorflow tf-keras

  Using cached h5py-3.11.0-cp312-cp312-win_amd64.whl.metadata (2.5 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
   ---------------------------------------- 0.0/385.2 MB ? eta -:--:--
   ---------------------------------------- 3.1/385.2 MB 15.4 MB/s eta 0:00:25
    --------------------------------------- 6.3/385.2 MB 14.9 MB/s eta 0:00:26
    --------------------------------------- 9.4/385.2 MB 15.4 MB/s eta 0:00:25
   - -------------------------------------- 13.1/385.2 MB 15.5 MB/s eta 0:00:24
   - -------------------------------------- 16.5/385.2 MB 15.8 MB/s eta 0:00:24
   -- ------------------------------------- 20.2/385.2 MB 15.9 MB/s eta 0:00:23
   -- ------------------------------------- 23.6/385.2 MB 16.2 MB/s eta 0:00:23
   -- ------------------------------------- 26.7/385.2 MB 16.1 MB/s eta 0:00:23
   --- ------------------------------------ 30.1/385.2 MB 16.2 MB/s eta 0:00:22
   --- ------------------------------------ 33.8/385.2 MB 16.3 MB/s eta 0:0

In [2]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

In [3]:
import torch
from PIL import Image
from datasets import load_dataset
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, Seq2SeqTrainer, Seq2SeqTrainingArguments
import gc

# Checking if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

c:\Users\Yassine\Downloads\E-Learning\fine_tuned_trocr_base_handwritten_OCRBench\ocr_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



cuda


## Dataset Loading and Model Preparation

In [4]:
# Loading the dataset
dataset = load_dataset("mychen76/invoices-and-receipts_ocr_v1")

# Model name
model_name = "microsoft/trocr-small-stage1"

# Processor & Model Loading
processor = TrOCRProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name).to(device)

# Set the decoder start token ID
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id

# Ensure pad_token_id is set
if processor.tokenizer.pad_token is None:
    processor.tokenizer.pad_token = processor.tokenizer.eos_token

# Set the pad_token_id in the model configuration
model.config.pad_token_id = processor.tokenizer.pad_token_id

c:\Users\Yassine\Downloads\E-Learning\fine_tuned_trocr_base_handwritten_OCRBench\ocr_env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-stage1 and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Image Preprocessing

In [5]:
# Function to resize images
def resize_image(image, max_width=1024, max_height=1024):
    image.thumbnail((max_width, max_height), Image.Resampling.LANCZOS)
    return image

In [6]:
# Preprocessing the dataset with resized images
def preprocess_batch(batch):
    images = [resize_image(img.convert("RGB")) if isinstance(img, Image.Image) else resize_image(Image.open(img).convert("RGB")) for img in batch['image']]

    # Get pixel values from the processor
    pixel_values = processor(images, return_tensors="pt").pixel_values.to(device)

    # Tokenizing the text data
    labels = processor.tokenizer(
        batch['parsed_data'],
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).input_ids.to(device)

    return {
        "pixel_values": pixel_values,
        "labels": labels
    }

## Custom Data Collator

In [7]:
# Custom Data Collator for VisionEncoderDecoderModel
def collate_fn(batch):
    # Ensure pixel_values are stacked as tensors
    pixel_values = torch.stack([torch.tensor(item["pixel_values"]) for item in batch])
    labels = torch.stack([torch.tensor(item["labels"]) for item in batch])
    
    return {
        "pixel_values": pixel_values,
        "labels": labels
    }

In [8]:
# Apply preprocessing
preprocessed_dataset = dataset.map(preprocess_batch, batched=True, remove_columns=['id', 'parsed_data', 'raw_data'])

# Dataset split
train_dataset = preprocessed_dataset['train']
eval_dataset = preprocessed_dataset['valid']
test_dataset = preprocessed_dataset['test']

Map:   0%|          | 0/2043 [03:57<?, ? examples/s]


ArrowMemoryError: realloc of size 3221225472 failed

In [ ]:
# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./trocr_finetuned_model",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    eval_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.tokenizer,
    data_collator=collate_fn,
)

In [ ]:
# Train the model
trainer.train()

# Save the model locally
model.save_pretrained("./trocr_finetuned_model")
processor.save_pretrained("./trocr_finetuned_model")